In [ ]:
from econml.iv.nnet import DeepIV

import keras
import numpy as np


import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from joblib import Parallel, delayed


In [ ]:

# Load dataset in the same way as HSIC-X
dataset_rpe1 = pd.read_csv("~/Documents/ethz/DIV/single_cell/dataset_rpe1.csv")

# Select relevant columns (same as HSIC-X)
interv_genes = dataset_rpe1.columns[:9].tolist()
train_data = dataset_rpe1[dataset_rpe1['interventions'].isin(interv_genes + ["non-targeting"])].copy()

# Convert intervention column to categorical
train_data['interventions'] = train_data['interventions'].astype('category')
train_data['Ztr'] = train_data.iloc[:, 10].astype('category')

# Get list of unique interventions (excluding "non-targeting")
unique_interventions = [g for g in train_data['interventions'].unique() if g != "non-targeting"]

# Convert `Ztr` to One-Hot Encoding AFTER removing one environment
encoder = OneHotEncoder(sparse_output=False)  # Drop first category for consistency
Ztr_encoded = encoder.fit_transform(train_data[['Ztr']])

# Extract features and target
Xtr = train_data.iloc[:, :9].values  # First 9 columns (gene expressions)
Ytr = train_data.iloc[:, 9].values   # 10th column (response variable)

# Convert to NumPy arrays for compatibility with the original method
data_train_length = Xtr.shape[0]

# Reformat to match the original method's expected structure
z = Ztr_encoded  # Encoded intervention variable
t = Xtr          # First 9 columns (gene expressions)
y = Ytr          # 10th column (response variable)
x = np.zeros(data_train_length)  # Dummy placeholder if needed for consistency


In [ ]:
z.shape, t.shape, y.shape, x.shape

In [ ]:
print(f'z (Instrument): {z}')
print(f't (Treatment): {t}')
print(f'y (Outcome): {y}')

In [ ]:
treatment_model = keras.Sequential([keras.layers.Dense(256, activation='relu', input_shape=(11,)),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(128, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])

response_model = keras.Sequential([keras.layers.Dense(256, activation='relu', input_shape=(10,)),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(128, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(64, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(32, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(1)])

In [ ]:
test_data_path = 'R/sec6.2/test_single_cell.csv'
test_data = np.genfromtxt(test_data_path, delimiter=',', skip_header=1)
X_test = test_data[:, 0:9].reshape(-1, 9)    

In [ ]:
# Function to train and predict DeepIV for a single run
def train_and_predict(run_id):
    print(f"Starting DeepIV Run {run_id + 1}/10...")

    keras_fit_options = { "epochs": 30,
                      "validation_split": 0.2,
                      "callbacks": [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]}

    deepIvEst = DeepIV(n_components = 10, # number of gaussians in our mixture density network
                m = lambda z, x : treatment_model(keras.layers.concatenate([z,x])), # treatment model
                h = lambda t, x : response_model(keras.layers.concatenate([t,x])),  # response model
                n_samples = 1, # number of samples to use to estimate the response
                use_upper_bound_loss = False, # whether to use an approximation to the true loss
                n_gradient_samples = 1, # number of samples to use in second estimate of the response (to make loss estimate unbiased)
                optimizer=keras.optimizers.Adam(learning_rate=0.0005), # Keras optimizer to use for training - see https://keras.io/optimizers/ 
                first_stage_options = keras_fit_options, # options for training treatment model
                second_stage_options = keras_fit_options) # options for training response model

    # Train DeepIV model
    deepIvEst.fit(Y=y, T=t, X=x, Z=z)

    # Predict on test data
    y_hat_deepIV = deepIvEst.predict(X_test, np.zeros(X_test.shape[0]))

    return y_hat_deepIV

# Run the 10 iterations in parallel
num_repeats = 10
results = Parallel(n_jobs=num_repeats)(
    delayed(train_and_predict)(i) for i in range(num_repeats)
)

# Convert results to DataFrame
df_results = pd.DataFrame(results).T  # Transpose to have runs as columns

# Rename columns to indicate run numbers
df_results.columns = [f'Run_{i+1}' for i in range(num_repeats)]

# Save the results as a CSV file
output_filename = 'results/deepIV_singlecell_10runs.csv'
df_results.to_csv(output_filename, index=False)

print(f"DeepIV predictions saved to {output_filename}")